## Preparation

In [1]:
!cp ../input/w2vu-cvsv-checkpoints-cpu6/checkpoint_best.pt .
!cp ../input/w2vu-cvsv-checkpoints-cpu6/checkpoint_last.pt .

cp: cannot stat '../input/w2vu-cvsv-checkpoints-cpu6/checkpoint_best.pt': No such file or directory


cp: cannot stat '../input/w2vu-cvsv-checkpoints-cpu6/checkpoint_last.pt': No such file or directory


In [2]:
%%capture
!conda install -c pykaldi pykaldi -y

In [3]:
%cd /tmp

/tmp


In [4]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...


remote: Enumerating objects: 28296, done.


remote: Total 28296 (delta 0), reused 0 (delta 0), pack-reused 28296
Receiving objects: 100% (28296/28296), 11.77 MiB | 20.99 MiB/s, done.


Resolving deltas: 100% (21291/21291), done.


In [5]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...


remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.


remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 19.07 MiB/s, done.


Resolving deltas: 100% (7956/7956), done.


In [6]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [7]:

%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [8]:
%%capture
%cd /tmp/kenlm
!python setup.py install
%cd /tmp

In [9]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [10]:
%cd /tmp/fairseq/

/tmp/fairseq


In [11]:
%%capture
!python setup.py install

In [12]:
%cd /tmp/fairseq/

/tmp/fairseq


In [13]:
os.environ['HYDRA_FULL_ERROR'] = '1'

In [14]:
%%capture
!pip install editdistance

## GAN

In [15]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir fairseq/config/model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.keep_last_epochs=20 \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [16]:
!bash rungan.sh

[2021-06-04 08:05:05,656][HYDRA] Launching 5 jobs locally
[2021-06-04 08:05:05,656][HYDRA] 	#0 : task.data=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/ task.text_data=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/ task.kenlm_path=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin checkpoint.no_epoch_checkpoints=False checkpoint.keep_last_epochs=20 checkpoint.save_dir=/kaggle/working common.seed=0


[2021-06-04 08:05:07,944][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': 'tb', 'wandb_project': None, 'azureml_logging': False, 'seed': 0, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': True, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overri

[2021-06-04 08:05:08,061][fairseq_cli.train][INFO] - Wav2vec_U(
  (discriminator): Discriminator(
    (net): Sequential(
      (0): Conv1d(45, 384, kernel_size=(6,), stride=(1,), padding=(5,))
      (1): SamePad()
      (2): Dropout(p=0.0, inplace=False)
      (3): Sequential(
        (0): Conv1d(384, 384, kernel_size=(6,), stride=(1,), padding=(5,))
        (1): SamePad()
        (2): Dropout(p=0.0, inplace=False)
        (3): GELU()
      )
      (4): Conv1d(384, 1, kernel_size=(6,), stride=(1,), padding=(5,))
      (5): SamePad()
    )
  )
  (segmenter): JoinSegmenter()
  (generator): Generator(
    (dropout): Dropout(p=0.1, inplace=False)
    (proj): Sequential(
      (0): TransposeLast()
      (1): Conv1d(512, 45, kernel_size=(4,), stride=(1,), padding=(2,), bias=False)
      (2): TransposeLast()
    )
  )
)
[2021-06-04 08:05:08,062][fairseq_cli.train][INFO] - task: UnpairedAudioText
[2021-06-04 08:05:08,062][fairseq_cli.train][INFO] - model: Wav2vec_U
[2021-06-04 08:05:08,062][fa

[2021-06-04 08:05:08,154][fairseq.data.extracted_features_dataset][INFO] - loaded 2019, skipped 0 samples
[2021-06-04 08:05:08,154][fairseq.tasks.unpaired_audio_text][INFO] - split valid has unpaired text? False
[2021-06-04 08:05:08,163][fairseq_cli.train][INFO] - training on 1 devices (GPUs/TPUs)
[2021-06-04 08:05:08,164][fairseq_cli.train][INFO] - max tokens per device = None and max sentences per device = 160
[2021-06-04 08:05:08,165][fairseq.trainer][INFO] - Preparing to load checkpoint /kaggle/working/checkpoint_last.pt
[2021-06-04 08:05:08,165][fairseq.trainer][INFO] - No existing checkpoint found /kaggle/working/checkpoint_last.pt
[2021-06-04 08:05:08,165][fairseq.trainer][INFO] - loading train data for epoch 1


[2021-06-04 08:05:08,222][fairseq.data.extracted_features_dataset][INFO] - loaded 2331, skipped 0 samples
[2021-06-04 08:05:08,222][fairseq.tasks.unpaired_audio_text][INFO] - split train has unpaired text? True
[2021-06-04 08:05:08,239][fairseq.data.data_utils][INFO] - loaded 2,328 examples from: /kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/train


...

[2021-06-04 16:56:00,596][valid][INFO] - {"epoch": 707, "valid_loss": "0.988", "valid_ntokens": "3039.79", "valid_nsentences": "144.214", "valid_lm_score_sum": "-88182.5", "valid_num_pred_chars": "47047", "valid_vocab_seen_pct": "0.900697", "valid_uer": "98.7593", "valid_weighted_lm_ppl": "77.2795", "valid_lm_ppl": "62.6934", "valid_wps": "16625.9", "valid_wpb": "3039.8", "valid_bsz": "144.2", "valid_num_updates": "11312", "valid_best_weighted_lm_ppl": "73.9191"}
[2021-06-04 16:56:00,599][fairseq.checkpoint_utils][INFO] - Preparing to save checkpoint for epoch 707 @ 11312 updates
[2021-06-04 16:56:00,601][fairseq.trainer][INFO] - Saving checkpoint to /kaggle/working/checkpoint707.pt


[2021-06-04 16:56:00,637][fairseq.trainer][INFO] - Finished saving checkpoint to /kaggle/working/checkpoint707.pt
[2021-06-04 16:56:00,665][fairseq.checkpoint_utils][INFO] - Saved checkpoint /kaggle/working/checkpoint707.pt (epoch 707 @ 11312 updates, score 77.27950826857929) (writing took 0.06564002400409663 seconds)
[2021-06-04 16:56:00,668][fairseq_cli.train][INFO] - end of epoch 707 (average epoch stats below)


[2021-06-04 16:56:00,671][train][INFO] - {"epoch": 707, "train_loss": "2.241", "train_ntokens": "145.688", "train_nsentences": "145.688", "train_temp": "1.137", "train_code_ppl": "8.528", "train_loss_code_pen": "0.249", "train_loss_smoothness": "1.698", "train_loss_dense_g": "2.491", "train_lm_score_sum": 0.0, "train_num_pred_chars": 0.0, "train_loss_grad_pen": "0.191", "train_loss_dense_d": "0.136", "train_loss_token_d": "0.111", "train_wps": "44.2", "train_ups": "0.3", "train_wpb": "145.7", "train_bsz": "145.7", "train_num_updates": "11312", "train_lr_discriminator": "0.0005", "train_lr_generator": "0.0004", "train_gnorm": "19.283", "train_clip": "87.5", "train_train_wall": "49", "train_wall": "31853"}


[2021-06-04 16:56:00,732][fairseq.trainer][INFO] - begin training epoch 708
[2021-06-04 16:56:00,733][fairseq_cli.train][INFO] - Start iterating over samples


[2021-06-04 16:56:47,234][fairseq_cli.train][INFO] - begin validation on "valid" subset
